In [1]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, Flatten, MaxPooling2D, Input, BatchNormalization, Dropout, Dense

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
 tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
train_path = 'train'
validation_path = 'val'
test_path = 'test'

def datagen_function(path, aug=False):
    if aug == True:
        generator = ImageDataGenerator(
            rescale = 1 / 255.0,
            rotation_range = 10,
            zoom_range = 0.1,
            horizontal_flip = True
        )
    else:
        generator = ImageDataGenerator(
            rescale = 1 / 255.0
        )
    return generator.flow_from_directory(
        path,
        target_size = (48,48),
        color_mode = 'rgb',
        shuffle = True,
        class_mode = 'categorical',
        batch_size = 64
    )

In [5]:
train_gen = datagen_function(train_path, True)
validation_gen = datagen_function(validation_path)
test_gen = datagen_function(test_path)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [6]:
model = Sequential()

base_resnet= tf.keras.applications.ResNet50(input_shape = (48,48,3), include_top= False, weights = 'imagenet')
base_resnet.summary()
base_resnet.trainable = False

model.add(base_resnet)
model.add(Flatten())

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='softmax'))

model.summary()

94765736/94765736 [==============================] - 3s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 54, 54, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 24, 24, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 24, 24, 64)   256         ['conv1_conv[0][0]']             
                          

In [7]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.0001, nesterov=True)

checkpoint = ModelCheckpoint('checkpoints/vgg16.h5', monitor='val_acc', save_best_only=True, save_freq="epoch")
reduce_lr2 = ReduceLROnPlateau(monitor='val_acc', mode='max', factor = 0.5, patience = 10, min_lr = 0.000001, verbose = 1)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics= 'acc')
history = model.fit(train_gen,validation_data = validation_gen, epochs = 100, callbacks = [reduce_lr2, checkpoint])

Epoch 1/100
449/449 [==============================] - 233s 491ms/step - loss: 2.0170 - acc: 0.2454 - val_loss: 1.8139 - val_acc: 0.2494 - lr: 0.0100
Epoch 2/100
449/449 [==============================] - 167s 372ms/step - loss: 1.7033 - acc: 0.3184 - val_loss: 1.6661 - val_acc: 0.3260 - lr: 0.0100
Epoch 3/100
449/449 [==============================] - 166s 369ms/step - loss: 1.5878 - acc: 0.3763 - val_loss: 1.5512 - val_acc: 0.4060 - lr: 0.0100
Epoch 4/100
449/449 [==============================] - 167s 372ms/step - loss: 1.4972 - acc: 0.4184 - val_loss: 1.4368 - val_acc: 0.4383 - lr: 0.0100
Epoch 5/100
449/449 [==============================] - 163s 362ms/step - loss: 1.4338 - acc: 0.4455 - val_loss: 1.5418 - val_acc: 0.4218 - lr: 0.0100
Epoch 6/100
449/449 [==============================] - 171s 380ms/step - loss: 1.3714 - acc: 0.4703 - val_loss: 1.3528 - val_acc: 0.4834 - lr: 0.0100
Epoch 7/100
449/449 [==============================] - 161s 358ms/step - loss: 1.3226 - acc: 0.4900 

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 54, 54, 3)    0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 24, 24, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 24, 24, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2]

In [6]:
model = Sequential()

base_resnet= tf.keras.applications.ResNet50(input_shape = (48,48,3), include_top= False, weights = 'imagenet')
base_resnet.summary()
for layer in base_resnet.layers[:170]:
    layer.trainable = False

model.add(base_resnet)
model.add(Flatten())

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='softmax'))

model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 54, 54, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 24, 24, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 24, 24, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

In [8]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.0001, nesterov=True)

checkpoint = ModelCheckpoint('checkpoints/resnet50.h5', monitor='val_acc', save_best_only=True, save_freq="epoch")
reduce_lr2 = ReduceLROnPlateau(monitor='val_acc', mode='max', factor = 0.5, patience = 10, min_lr = 0.000001, verbose = 1)

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics= 'acc')
history = model.fit(train_gen,validation_data = validation_gen, epochs = 100, callbacks = [reduce_lr2, checkpoint])

Epoch 1/100
449/449 [==============================] - 186s 406ms/step - loss: 1.8070 - acc: 0.2421 - val_loss: 1.7735 - val_acc: 0.2639 - lr: 0.0100
Epoch 2/100
449/449 [==============================] - 42s 94ms/step - loss: 1.7730 - acc: 0.2649 - val_loss: 1.8334 - val_acc: 0.2313 - lr: 0.0100
Epoch 3/100
449/449 [==============================] - 44s 99ms/step - loss: 1.7542 - acc: 0.2811 - val_loss: 1.7393 - val_acc: 0.2901 - lr: 0.0100
Epoch 4/100
449/449 [==============================] - 42s 94ms/step - loss: 1.7467 - acc: 0.2830 - val_loss: 1.7526 - val_acc: 0.2705 - lr: 0.0100
Epoch 5/100
449/449 [==============================] - 45s 101ms/step - loss: 1.7353 - acc: 0.2911 - val_loss: 1.7137 - val_acc: 0.2951 - lr: 0.0100
Epoch 6/100
449/449 [==============================] - 42s 94ms/step - loss: 1.7289 - acc: 0.2980 - val_loss: 1.7194 - val_acc: 0.2820 - lr: 0.0100
Epoch 7/100
449/449 [==============================] - 43s 96ms/step - loss: 1.7219 - acc: 0.2984 - val_loss:

In [9]:
for layer in model.layers[:170]:
    layer.trainable = True

In [10]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.0001, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics= 'acc')
history = model.fit(train_gen,validation_data = validation_gen, epochs = 100, callbacks = [reduce_lr2, checkpoint])

Epoch 1/100
449/449 [==============================] - 76s 153ms/step - loss: 1.5894 - acc: 0.3852 - val_loss: 1.8339 - val_acc: 0.2385 - lr: 0.0100
Epoch 2/100
449/449 [==============================] - 71s 157ms/step - loss: 1.3057 - acc: 0.4998 - val_loss: 1.3383 - val_acc: 0.4932 - lr: 0.0100
Epoch 3/100
449/449 [==============================] - 74s 164ms/step - loss: 1.2116 - acc: 0.5370 - val_loss: 1.6084 - val_acc: 0.4324 - lr: 0.0100
Epoch 4/100
449/449 [==============================] - 73s 162ms/step - loss: 1.1651 - acc: 0.5611 - val_loss: 1.2492 - val_acc: 0.5135 - lr: 0.0100
Epoch 5/100
449/449 [==============================] - 67s 150ms/step - loss: 1.1226 - acc: 0.5768 - val_loss: 1.4619 - val_acc: 0.4216 - lr: 0.0100
Epoch 6/100
449/449 [==============================] - 71s 159ms/step - loss: 1.1030 - acc: 0.5826 - val_loss: 1.1517 - val_acc: 0.5726 - lr: 0.0100
Epoch 7/100
449/449 [==============================] - 71s 158ms/step - loss: 1.0513 - acc: 0.6049 - val_l